# Quick calculations in Astronomy

## Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as sc
import astropy.constants as ast
import astropy.units as unidades
from IPython.display import display, Math, Latex
from icecream import ic
import time
T0 = time.time()

## Constants and Converions 

| Nome variável | O que representa |                    
|---------------|------------------|                    
| anos | 1 ano em segundos |                          
| au | 1 unidade astronómica em metros |            
| pc | 1 parsec em metros |
| rTerraEquador | raio Terra Equador em metros |
| rTerra Polos | raio Terra Polos metros |
| mTerra | massa da Terra em kilos |
| m_Jup | massa de Jupiter em kilos |
| mSol | massa do Sol em kilos |
| mMercurio | massa Mercúrio em kilos |
| LSol | Luminosidade do Sol em Watts |

| Nome variável   | O que representa |
|---------------  |------------------|
| g_para_rad      | $x$ graus    * g_para_rad = $x$ radianos |
| seg_para_rad    | $x$ segundos * seg_para_rad = $x$ rad |
| rad_para_arcsec | $x$ radianos * rad_para_arcsec = $x$ arcsec |
| erg_para_J      | $x$ ergs     * erg_para_J = $x$ Joule |

In [2]:
"""
Usar sempre unidades SI
"""
# Constantes:
anos = 365.25*24*60*60              # s
au = 1.495978707 * 10 ** 11         # m
pc = 206264.8 * au                  # m
rTerraEquador = 6.3781 * 10 ** 6    # m
rTerraPolos = 6.3568 * 10 ** 6      # m
mTerra = 5.972365 * 10 ** 24        # kg
M_Jup = 1.898*10**27                # kg
mSol = 1.988475 * 10 ** 30          # kg
mMercurio = 3.3011 * 10 ** 23       # kg
LSol = 3.828 * 10 ** 26             # W

# Conversões
g_para_rad = np.pi/180              # rad
seg_para_rad = np.pi/(180 * 3600)   # rad
rad_para_arcsec = 206265            # arcesc
erg_para_J = 10**-7                 # J ---> 1 erg = 10**-7 J



## Functions

| Astrofísica                                                   | Funções                           |
|---------------------------------------------------------------|-----------------------------------|
| 3ª Lei Kepler:                                                | T(m1,m2,a)                      |
| Velocidade Apocentro:                                         | v_A(m1,m2,a,e)                  |
| Velocidade Pericentro:                                        | v_P(m1,m2,a,e)                  |
| Distância a um Foco em função do ângulo:                      | r_Teta(teta,teta_0,a,e)         |
| Velocidade Orbial num ponto:                                  | v_Orbital(m1,m2,r,a)            |
| Massa de cada estrela num binário:                            | m_Binario(a1,a2,T)              |
| magnitude aparente:                                           | m_aparente(m0,b_estrela,b0)     |
| Magnitude absoluta:                                           | M_absoluta(mAparente,d)         |
| magnitude no topo atmosfera:                                  | m_topo_atm(mz,m0,z)             |
| Profundidade ótica:                                           | Profundidade_otica(m0,mEstrela) |
| Luminosidade duma estrela:                                    | L_estrela1(Mbol1,Mbol2,L2)      |
| Equação de Boltzman:                                          | eq_Boltzman(i,n,X,T)            |
| Equação de Saha:                                              | eq_Saha(Z2,Z1,T,Pe,X2)          |
| Semi-amplitude de RV:                                         | K(m_star, m_pl, P_pl, e_pl, i, show_eq=True)  |
| Roche limit:                                                  | Roche_limit(R_M, rho_M, rho_m, show_eq=True)  |
| Probability of planet crossing in front of star:              | Prob_transit(R_star, a_pl)  |
| Amplitude of transmission spectroscopy:                       | Amp_trans_spec(R_p, R_star, H, show_eq=True)  |
| Photometric (maximum) amplitude of the phase curve variations:| Amp_photometric_signal(R_p, a, Ag, phi_alpha=1, show_eq=True)  |
| Astrometric amplitude of around the Center of Mass:           | Amp_astrometric_signal( M_pl, M_star, a, D_system, show_eq=True)  |


###
| Outros                                                  | Funções                         |
|---------------------------------------------------------|---------------------------------|
| Ajuste linear / mínimos quadrados:                      | AjusteLinear(x,y)               |
| Notação científica:                                     | n_cientifica(valor,alg_sig)     |

In [3]:
################# Funções para cálculos em Astrofísica Estrelar ##############################

def T(m1,m2,a):
    return np.sqrt( 4 * np.pi**2 * a**3 / (sc.G * (m1+m2)) ),"s"

def v_A(m1,m2,a,e):
    return np.sqrt( sc.G * (m1 + m2) * (1 - e) / a / (1 + e)  ) , "m/s"

def v_P(m1,m2,a,e):
    return np.sqrt( sc.G * (m1 + m2) * (1 + e) / a / (1 - e) ) , "m/s"

def r_Teta(teta,teta_0,a,e):
    print("Aviso: usa o ângulo em graus")
    return a * (1-e**2) / (1 + e*np.cos(teta*g_para_rad - teta_0*g_para_rad)), 'msm unidades que "a" '

def v_Orbital(m1,m2,r,a):
    return np.sqrt( sc.G*(m1+m2)*(2/r - 1/a) ),"m/s"

def m_Binario(a1,a2,T):
    m1 = 4*np.pi**2/sc.G * (a1+a2)**2/T**2 * a2
    m2 = m1 * a1/a2
    return print( "m1 = {}\nm2 = {}".format(m1,m2) )

def m_aparente(m0,b_estrela,b0):
    return m0 - 2.5 * np.log10( b_estrela/b0 )

def M_absoluta(mAparente,d):
    return mAparente + 5 - 5 * np.log10( d )

def m_topo_atm(mz,m0,z):
    print("Aviso: usa o ângulo z em radianos. ")
    return (mz - m0 * 1/np.cos(z)) / (1 - 1/np.cos(z))

def Profundidade_otica(m0,mEstrela):
    return (m0 - mEstrela) / 1.086

def L_estrela1(Mbol1,Mbol2,L2):
    L1 = L2 * 10 **((Mbol2 - Mbol1)/2.5)
    return print( "L1  = {} = {} * L2".format(L1,L1/L2) )

def eq_Boltzman(i,n,X,T):
    display(Math(r"Aviso: E_n = - \frac{X}{n^2}"))
    """ retorna Ni / Nn"""
    return (i**2 / n**2) * 10**-(-X*(1/i**2 - 1/n**2) * 5040/T)

def eq_Saha(Z2,Z1,T,Pe,X2):
    return 2 * Z2/Z1 * T**2.5 / Pe * 10**(-X2*5040/T - 0.48)


In [4]:
############# Functions made for Planetary Systems #############################

def K(m_star, m_pl, P_pl, e_pl, i, show_eq=True):
    if show_eq == True:
        display(Math(r"K = \left(\frac{2\pi G}{P_{orb}}\right)^{1/3} \,\, \frac{m_{pl}}{(m_\star + m_{pl})^{2/3}} \frac{\sin i}{\sqrt{1-e^2}}"))
    return (2*np.pi*sc.G / P_pl)**(1/3) * m_pl/(m_star + m_pl)**(2/3) * np.sin(i) / np.sqrt(1-e_pl**2)

def Roche_limit(R_M, rho_M, rho_m, show_eq=True):
    Roche_Radius = 2.44 * R_M * (rho_M / rho_m)**(1/3)
    if show_eq == True:
        display(Math(r"R_R \simeq 2,44\,R_M\left( \frac{\rho_M}{\rho_m} \right)^{1/3}"))
        display(Latex(r"where $M$ represents the more massive body, and $m$ the less massive one"))
    return Roche_Radius

def Prob_transit(R_star, a_pl):
    return R_star / a_pl

def Amp_trans_spec(R_p, R_star, H, show_eq=True):
    if show_eq == True:
        display(Math(r"\Delta \simeq \frac{2R_{pl}H}{R_\star^{\,2}}   \quad ,   \quad \Delta_{\text{Full Height}} \simeq \frac{2R_{pl}5H}{R_\star^{\,2}}"))
    Delta = 2 * R_p * H / R_star**2
    Delta_Full_height = 2 * R_p * (5*H) / R_star**2
    return Delta, Delta_Full_height

def Amp_photometric_signal(R_p, a, Ag, phi_alpha=1, show_eq=True):
    if show_eq == True:
        display(Math(r"\frac{F_P}{F_*} = \frac{R_P^2}{a^2} A_g \, \phi(\alpha)"))
        display(Latex(r"A amplitude é máxima quando $\phi(\alpha)=1$"))
    Phot_Max_Amp = R_p**2 / a**2 * Ag * phi_alpha
    return Phot_Max_Amp

def Amp_astrometric_signal( M_pl, M_star, a, D_system, show_eq=True):
    if show_eq == True:
        display(Math(r"\begin{equation}\Delta \theta = \frac{M_p}{M_\star} \frac{a_{p}}{d_\star} \;\; [\text{radian}] \quad = \frac{M_p}{M_\star} \frac{a_{p}}{d_\star} \times 206265 \;\; [\text{arcsec}]\end{equation}"))
        display(Latex(r"where $d_\star$ is the distance from the star to our Solar System."))
    Delta_Theta = M_pl / M_star * a / D_system
    return Delta_Theta, Delta_Theta*rad_para_arcsec



In [5]:
################## Funções para cálculos matemáticos ###########################

def AjusteLinear(x,y): #método mínimos quadrados
    print("Aviso: x , y são arrays com o mesmo comprimento")
    N = len(x)

    Ex = 1/N * sum(x)
    Ey = 1/N * sum(y)
    Exx = 1/N * sum(x**2)
    Exy = 1/N * np.dot(x, y)
    m = (Exy - Ex * Ey)/(Exx - Ex**2)
    b = (Exx * Ey - Ex * Exy)/(Exx - Ex**2)
    
    # Gráfico
    y_novo = np.empty(N, float)
    for i in range(0, N):
        y_novo[i] = m * x[i] + b
    plt.plot(x, y, 'ko')
    plt.plot(x, y_novo, 'k')
    plt.xlabel("x")
    plt.ylabel("y")
    plt.show()
    print("y = m * x + b    ou    x = (y-b) / m")
    return print(f"m = {m}\nb = {b}")
# x_exemplo = np.array([5560,5310] )
# y_exemplo = np.array([5.1 , 5.5])
# AjusteLinear( x_exemplo , y_exemplo)

def n_cientifica(valor,alg_sig):
    if valor > 1 or valor < -1:
        expoente = len(str(int(valor))) - 1
        n_final = int( int(valor) / (10**(expoente-alg_sig+1)) )
        resposta = str(n_final)[:1] + "," + str(n_final)[1:]
    else:
        i = 0
        while int(valor * 10**i) < 1 and int(valor * 10**i) > -1 :
            expoente = -i -1
            i += 1
        n_final = int(valor * 10**(-expoente+alg_sig-1))
        resposta = str(n_final)[:1] + "," + str(n_final)[1:]
    return "{} * 10^{}".format(resposta,expoente)

## Programar

In [18]:
N = 4/3*np.pi*55*(4.6*10**22)**3
b = N * 1.67*10**(-27)
c = b / ast.M_sun.value
ic(N,b, c)

print(ast.k_B)

e_ter = 3/2*ast.k_B.value * 7*10**7 * N
l = 1.5*10**36
ic(e_ter/l  /anos)

ic| N: 2.242460458551984e+70
    b: 3.7449089657818136e+43
    c: 18833687264221.47
ic| e_ter/l  /anos: 686755505091.4952


  Name   = Boltzmann constant
  Value  = 1.380649e-23
  Uncertainty  = 0.0
  Unit  = J / K
  Reference = CODATA 2018


686755505091.4952

### Tempo

In [6]:
#################################### Tempo total que o programa demora a correr ############################################
t1=time.time()
print("O código do notebook inteiro demorou um total de {} min e {} segundos a correr.".format((t1-T0)//60,int((t1-T0)%60)))

O código do notebook inteiro demorou um total de 0.0 min e 0 segundos a correr.
